In [42]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

import pandas as pd
import numpy as np

In [101]:
train_df = pd.read_csv('./data/iris.csv')
np.random.shuffle(train_df.values)

print(train_df.head())

train_x = np.column_stack((train_df.SepalLengthCm.values, train_df.SepalWidthCm.values, train_df.PetalLengthCm.values, train_df.PetalWidthCm.values))

   Id  SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm      Species
0   1            5.1           3.5            1.4           0.2  Iris-setosa
1   2            4.9           3.0            1.4           0.2  Iris-setosa
2   3            4.7           3.2            1.3           0.2  Iris-setosa
3   4            4.6           3.1            1.5           0.2  Iris-setosa
4   5            5.0           3.6            1.4           0.2  Iris-setosa


In [102]:
# LABEL ENCODING

print(train_df.Species.unique())

species_dict = {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}
train_df['Species'] = train_df.Species.apply(lambda x: species_dict[x])

print('\nAfter label encoding:')
print(train_df.Species.unique())

['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']

After label encoding:
[0 1 2]


In [ ]:
# LABEL ENCODING - using sklearn

# from sklearn import preprocessing
# label_encoder = preprocessing.LabelEncoder()
# train_df['Species']= label_encoder.fit_transform(train_df['Species'])
# train_df['Species'].unique()

In [103]:
model = keras.Sequential([
	keras.layers.Dense(64, input_shape=(4,), activation='relu'),
    # keras.layers.Dropout(0.2),									# % of neurons (of previous layer) that will be dropped
    keras.layers.Dense(64, activation='relu'),
	keras.layers.Dense(64, activation='relu'),						# adding layer, because quadratic example is more complex
	keras.layers.Dense(3, activation='sigmoid')])

In [104]:
optimizers = {'Adadelta': None, 'Adagrad': None, 'Adam': None, 'RMSprop': None, 'SGD': None}

early_stopping = EarlyStopping(monitor='accuracy', patience=3, restore_best_weights=True)

best_optimizer = None
best_accuracy = 0
best_optimizer_weights = None

for i, _ in optimizers.items():

    print(f'\n{i}')
    model.compile(optimizer=i, loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    results = model.fit(train_x, train_df.Species.values, batch_size=4, epochs=10, callbacks=[early_stopping])

    current_accuracy = max(results.history['accuracy'])
    print(f'\nBest Accuracy: {current_accuracy:.4f} at epoch {results.history["accuracy"].index(current_accuracy) + 1}')
    optimizers[i] = round(current_accuracy, 4)

    if current_accuracy > best_accuracy:
        best_optimizer = i
        best_accuracy = current_accuracy
        best_optimizer_weights = model.get_weights()

    
print('\n\n------------')
print(f'Optimizers: {optimizers}')
print(f'Best optimizer: {best_optimizer}')

model.set_weights(best_optimizer_weights)


Adadelta
Epoch 1/10


/home/hubert/.local/lib/python3.10/site-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


38/38 [==============================] - 1s 5ms/step - loss: 1.2889 - accuracy: 0.3333
Epoch 2/10
38/38 [==============================] - 0s 5ms/step - loss: 1.2852 - accuracy: 0.3333
Epoch 3/10
38/38 [==============================] - 0s 5ms/step - loss: 1.2819 - accuracy: 0.3333
Epoch 4/10
38/38 [==============================] - 0s 5ms/step - loss: 1.2785 - accuracy: 0.3333

Best Accuracy: 0.3333 at epoch 1

Adagrad
Epoch 1/10
38/38 [==============================] - 1s 5ms/step - loss: 1.1368 - accuracy: 0.5133
Epoch 2/10
38/38 [==============================] - 0s 5ms/step - loss: 1.0204 - accuracy: 0.6333
Epoch 3/10
38/38 [==============================] - 0s 5ms/step - loss: 0.9760 - accuracy: 0.4800
Epoch 4/10
38/38 [==============================] - 0s 5ms/step - loss: 0.9453 - accuracy: 0.5733
Epoch 5/10
38/38 [==============================] - 0s 5ms/step - loss: 0.9182 - accuracy: 0.5867

Best Accuracy: 0.6333 at epoch 2

Adam
Epoch 1/10
38/38 [============================

In [78]:
test_df = pd.read_csv('./data/iris.csv')
test_x = np.column_stack((test_df.SepalLengthCm.values, test_df.SepalWidthCm.values, test_df.PetalLengthCm.values, test_df.PetalWidthCm.values))

test_df['Species'] = test_df.Species.apply(lambda x: species_dict[x])

print("EVALUATION")
evaluation = model.evaluate(test_x, test_df.Species.values)

EVALUATION
5/5 [==============================] - 0s 5ms/step - loss: 0.1048 - accuracy: 0.9733


/home/hubert/.local/lib/python3.10/site-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


In [66]:
random_index = np.random.randint(0, len(test_df))

random_sample = test_x[random_index].reshape(1, -1)     # reshaping to (1, 4) as there are 4 features

ground_truth = test_df.iloc[random_index]['Species']
ground_truth_species = None


prediction_probs = model.predict(random_sample)
prediction_probs = np.round(prediction_probs, decimals=4)

predicted_label = np.argmax(prediction_probs)
predicted_label_species = None

for key, value in species_dict.items():
    if value == ground_truth:
        ground_truth_species = key
    if value == predicted_label:
        predicted_label_species = key
    if ground_truth_species != None and predicted_label_species != None:
        break

# Print the ground truth and prediction
print(f"Ground Truth: {int(ground_truth)} ({ground_truth_species})")
print(f"Prediction: {predicted_label} ({predicted_label_species})\n(Probability distribution: {prediction_probs})")

1/1 [==============================] - 0s 37ms/step
Ground Truth: 2 (Iris-virginica)
Prediction: 2 (Iris-virginica)
(Probability distribution: [[0.0082 0.7565 0.9823]])
